## Named Entity Recognition


#### Постановка задачи «sequence labeling»:



* Дан корпус текстов $D$
* Каждый текст представляет собой последовательность токенов
* Каждому токену присвоена метка из некоторого множества $V$

В зависимости от множества меток $V$ получаем разные типы подзадач. Например:
* если $V$ - множество частей речи, то это задача ***POS***-теггинга
* если $V$ - множество типов именованных сущностей, то это задача ***NER***

Именованная сущность - любой фрагмент текста, обозначающий некоторый интересный объект.

### На правилах: Наташа

In [2]:
!pip3 install natasha

    100% |████████████████████████████████| 778kB 1.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/37/64/d6abf637228bed6b0249b522f588d19dca9f09ab65db13bef41096f51889/yargy-0.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/40/0b1db94fdfd71353ae67ec444ff28e0a7ecc25212d1cb94c291b6cd226f9/backports.functools_lru_cache-1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a3/33/fff9675c68b5f6c63ec8c6e6ff57827dda28a1fa5b2c2d727dffff92dd47/pymorphy2-0.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/84/ff1ce2071d4c650ec85745766c0047ccc3b5036f1d03559fd46bb38b5eeb/DAWG_Python-0.7.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/51/2465fd4f72328ab50877b54777764d928da8cb15b74e2680fc1bd8cb3173/pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 19.2.3 is available.
You should consider upgrading v

In [1]:
text = 'КПРФ решила передать место Алферова в Госдуме Грудинину'

In [2]:
from natasha import *

Имена:

In [11]:
names_extractor(text)

Matches('КПРФ решила передать место Алферова в Госдуме Грудинину',
        [Match([MorphTagToken('Алферова',
                              [27, 35),
                              'RU',
                              'I',
                              [Form('алферов',
                                    Grams(NOUN,Sgtm,Surn,anim,gent,masc,sing)),
                               Form('алферов',
                                    Grams(NOUN,Sgtm,Surn,accs,anim,masc,sing)),
                               Form('алферов',
                                    Grams(NOUN,Sgtm,Surn,anim,femn,nomn,sing))])],
               [27, 35)),
         Match([MorphTagToken('Грудинину',
                              [46, 55),
                              'RU',
                              'I',
                              [Form('грудинин',
                                    Grams(NOUN,Sgtm,Surn,anim,datv,masc,sing)),
                               Form('грудинин',
                                    Grams(NOUN,Sgtm,Surn,accs,anim,femn,sing))])],
               [46, 55))])

In [7]:
names_extractor = NamesExtractor()
for match in names_extractor(text):
    print(match.span, match.fact)

[27, 35) Name(first=None, middle=None, last='алферов', nick=None)
[46, 55) Name(first=None, middle=None, last='грудинин', nick=None)


Всё вместе:

In [8]:
all_extractors = [NamesExtractor, SimpleNamesExtractor, DatesExtractor, MoneyExtractor,
                  MoneyRateExtractor, MoneyRangeExtractor, LocationExtractor, AddressExtractor,
                  OrganisationExtractor, PersonExtractor]

In [9]:
extractors = [extractor() for extractor in all_extractors]

In [10]:
for extractor in extractors:
    matches = extractor(text)
    for match in matches:
        print(match.span, match.fact)

[27, 35) Name(first=None, middle=None, last='алферов', nick=None)
[46, 55) Name(first=None, middle=None, last='грудинин', nick=None)
[27, 35) Name(first=None, middle=None, last='алферов', nick=None)
[36, 37) Name(first=None, middle=None, last='в', nick=None)
[46, 55) Name(first=None, middle=None, last='грудинин', nick=None)
[0, 4) Organisation(name='КПРФ')
[38, 45) Organisation(name='Госдуме')
[27, 35) Person(position=None, name=Name(first=None, middle=None, last='алферов', nick=None))
[46, 55) Person(position=None, name=Name(first=None, middle=None, last='грудинин', nick=None))


### На машинном обучении: Spacy 

In [15]:
# !python3 -m spacy download en_core_web_sm

In [16]:
# !pip3 install html5lib

In [17]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [18]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


Выкачаем статью и найдём в ней именованные сущности, выведем их число:

In [6]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [7]:
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)

In [8]:
len(article.ents)

173

Выведем число встреченных сущностей каждого типа:

In [9]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'CARDINAL': 5,
         'DATE': 23,
         'FAC': 1,
         'GPE': 16,
         'LAW': 1,
         'NORP': 2,
         'ORDINAL': 1,
         'ORG': 39,
         'PERSON': 82,
         'PRODUCT': 3})

Выведем текст с подсвеченными сущностями разных типов:

In [10]:
sentences = [x for x in article.sents]
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

#### [BiLSTM-CRF](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling)

![bilstm_crf](bilstm_crf_model.png)

Основные шаги алгоритма:
* Получить предобученные эмбеддинги слов коллекции (***word2vec***, ***GloVe***)
$$$$
* Обучить символьные эмбеддинги (***char-BiLSTM***, ***char-CNN***)
$$$$
* Составить для каждого слова морфологические/синтаксические признаки (***POS***-тег, роль в предложении и т.п.)
$$$$
* Объединить всё это и подать на вход основной сети (***BiLSTM***)
$$$$
* Выходы $h_t$ для всех слов предложения подавать на вход классификатору,
который будет предсказывать NER-тег (***SoftMax***, ***CRF***)